In [2]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import sys

In [3]:
vest_or_16 = gp.read_file("./raw-from-source/VEST/or_2016/or_2016.shp")

In [5]:
vest_or_16.columns

Index(['STATE', 'COUNTY', 'PRECINCT', 'NAME', 'G16PREDCLI', 'G16PRERTRU',
       'G16PRELJOH', 'G16PREGSTE', 'G16PREOWRI', 'G16USSDWYD', 'G16USSRCAL',
       'G16USSLLIN', 'G16USSGNAV', 'G16USSWLEW', 'G16USSIREY', 'G16USSOWRI',
       'G16GOVDBRO', 'G16GOVRPIE', 'G16GOVLFOS', 'G16GOVCAUE', 'G16GOVITHO',
       'G16GOVOWRI', 'G16ATGDROS', 'G16ATGRCRO', 'G16ATGLHED', 'G16ATGOWRI',
       'G16SOSDAVA', 'G16SOSRRIC', 'G16SOSLDUR', 'G16SOSGZUN', 'G16SOSCMAR',
       'G16SOSIWEL', 'G16SOSOWRI', 'G16TREDREA', 'G16TRERGUD', 'G16TREPHEN',
       'G16TREITEL', 'G16TREOWRI', 'geometry'],
      dtype='object')

Election results from the Oregon Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-or/). Results from Polk, Tillamook, Wallowa, and Wasco counties were corrected with the respective county reports.

Precinct shapefiles for the 2016 election were obtained from the following counties: Benton, Columbia, Coos, Crook, Curry, Deschutes, Douglas, Harney, Jackson, Josephine, Klamath, Lane, Linn, Malheur, Morrow, Multnomah, Polk, Sherman, Tillamook, Umatilla, Union, Wallowa, Wasco, Washington, Yamhill.

Precinct shapefiles for the 2016 election were provided by the Oregon Secretary of State's office for the following counties: Clatsop, Hood River, Jefferson, Marion. The Marion County shapefile included numerous gaps that were assigned based on the Precinct Split Summary report from the county clerk's office. Precincts along the Salem/Kaiser border were revised with the corrected municipal boundaries from the Marion County 2019 precinct shapefile.

The 2016 precinct shapefile for Clackamas County was provided by the Metro Regional Government Data Resource Center.

Precinct boundaries for Grant County and Wheeler County are defined by school district boundaries. Precinct shapefiles were produced to match PDF maps provided by the respective counties using the 2016 Oregon Education Boundaries shapefile obtained from the Oregon Spatial Data Library.

Precinct shapefiles for the following counties were produced based on PDF maps provided by the respective counties: Baker, Gilliam, Lake, Lincoln. Boundaries defined by the PLSS grid were produced with the Oregon Public Land Survey Quarter-Quarter Reference Grid shapefile obtained from the Oregon spatial Data Library. Municipal boundaries were produced from the Oregon Department of Transportion 2016 City Limits shapefile obtained from the Oregon Spatial Data Library. Street boundaries are produced from the U.S. Census Bureau's census block shapefiles.

PDF maps obtained from Lake County are approximate precinct boundaries drawn on the PLSS grid for county precincts and on the street map for Lakeview city precincts. These boundaries were adjusted to match address range assignments in the Lake County Address Library Report which serves as the legal definition of the precincts according to the Lake County Clerk. Precinct divisions for roads that cross multiple rural precincts are generally defined by zip codes. Address ranges were further identified based on the Lake County tax lot parcel viewer.

Rural precinct boundaries in the shapefiles provided by Coos County and Union County were revised to match the updated PLSS cadastral grid in the Oregon GIS Framework obtained from the Oregon Spatial Data Library. 

Municipal precinct boundaries in the shapefiles provided by Tillamook County and Umatilla County were revised to reverse annexations that were not yet in effect for the November 2016 general election. 

The boundaries of precincts 14, 37, 45, 49 in the Deschutes County shapefile were revised to match legislative district assignments.

Precinct numbers in the Columbia County, Josephine County, and Wallowa County shapefiles were corrected to match the Oregon Secretary of State voter file.

G16PREDCLI - Hillary Clinton (Democratic Party)
G16PRERTRU - Donald Trump (Republican Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREGSTE - Jill Stein (Green Party)
G16PREOWRI - Write-in Votes

G16USSDWYD - Ron Wyden (Democratic Party)
G16USSRCAL - Mark Callahan (Republican Party)
G16USSLLIN - Jim Lindsay (Libertarian Party)
G16USSGNAV - Eric Navickas (Pacfic Green Party)
G16USSWLEW - Shanti Lewallen (Working Families Party)
G16USSIREY - Steven Cody Reynolds (Independent Party)
G16USSOWRI - Write-in Votes

G16GOVDBRO - Kate Brown (Democratic Party and Working Families Party (fusion candidate))
G16GOVRPIE - Bud Pierce (Republican Party)
G16GOVLFOS - James Foster (Libertarian Party)
G16GOVCAUE - Aaron Donald Auer (Constitution Party)
G16GOVITHO - Cliff Thomason (Independent Party)
G16GOVOWRI - Write-in Votes

G16ATGDROS - Ellen Rosenblum (Democratic Party and Working Families Party (fusion candidate))
G16ATGRCRO - Daniel Zene Crowe (Republican Party)
G16ATGLHED - Lars D. H. Hedbor (Libertarian Party)
G16ATGOWRI - Write-in Votes

G16SOSDAVA - Brad Avakian (Democratic Party, Working Families, and Progressive Party (fusion candidate))
G16SOSRRIC - Dennis Richardson (Republican Party)
G16SOSLDUR - Sharon L. Durbin (Libertarian Party)
G16SOSGZUN - Alan Zundel (Pacific Green Party)
G16SOSCMAR - Michael Marsh (Constitution Party)
G16SOSIWEL - Paul Damian Wells (Independent Party)
G16SOSOWRI - Write-in Votes

G16TREDREA - Tobias Read (Democratic Party)
G16TRERGUD - Jeff Gudman (Republican Party)
G16TREPHEN - Chris Henry (Progressive Party and Pacific Green Party (fusion candidate))
G16TREITEL - Chris Telfer (Independent Party)
G16TREOWRI - Write-in Votes

In [10]:
file_list = os.listdir("./raw-from-source/Open_Elections/2016/counties")
li = []
for file in file_list:
    #print(file)
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/Open_Elections/2016/counties/"+file
        file_prev = pd.read_csv(file_ref)
        if 'precincts' in file_prev.columns:
            file_prev['precinct'] = file_prev['precincts']
     
        file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
        li.append(file_prev)
election_results = pd.concat(li, axis=0, ignore_index=True)
display(election_results)

,county,precinct,office,district,party,candidate,votes
0,Crook,1,U.S. Senate,NaN,IND,Steven C Reynolds,22
1,Crook,1,President,NaN,REP,Donald J Trump / Mike Pence,309
2,Crook,1,U.S. House,2.0,REP,Greg Walden,354
3,Crook,1,President,NaN,DEM,Hillary Clinton / Tim Kaine,122
4,Crook,1,U.S. House,2.0,DEM,James (Jim) Crary,104
...,...,...,...,...,...,...,...
72422,Sherman,3,State House,57.0,NaN,Under Votes,55
72423,Sherman,4,State House,57.0,NaN,Over Votes,0
72424,Sherman,4,State House,57.0,NaN,Under Votes,31
72425,Sherman,5,State House,57.0,NaN,Over Votes,0


In [11]:
print(election_results)

        county precinct       office  district party  \
0        Crook        1  U.S. Senate       NaN   IND   
1        Crook        1    President       NaN   REP   
2        Crook        1   U.S. House       2.0   REP   
3        Crook        1    President       NaN   DEM   
4        Crook        1   U.S. House       2.0   DEM   
...        ...      ...          ...       ...   ...   
72422  Sherman        3  State House      57.0   NaN   
72423  Sherman        4  State House      57.0   NaN   
72424  Sherman        4  State House      57.0   NaN   
72425  Sherman        5  State House      57.0   NaN   
72426  Sherman        5  State House      57.0   NaN   

                         candidate  votes  
0                Steven C Reynolds     22  
1      Donald J Trump / Mike Pence    309  
2                      Greg Walden    354  
3      Hillary Clinton / Tim Kaine    122  
4                James (Jim) Crary    104  
...                            ...    ...  
72422              

In [14]:
election_results["office"].unique()

array(['U.S. Senate', 'President', 'U.S. House', 'Governor',
       'Attorney General', 'Secretary of State', 'State Treasurer',
       'State Senate', 'State House'], dtype=object)

In [16]:
election_results = election_results[election_results["office"].isin(['U.S. Senate', 'President', 'Governor',
       'Attorney General', 'Secretary of State', 'State Treasurer'])]


In [31]:
election_results["pivot_id"] = election_results["county"].astype(str) +"-"+ election_results["precinct"].astype(str)

In [32]:
election_results

,county,precinct,office,district,party,candidate,votes,pivot_id
0,Crook,1,U.S. Senate,NaN,IND,Steven C Reynolds,22,Crook-1
1,Crook,1,President,NaN,REP,Donald J Trump / Mike Pence,309,Crook-1
3,Crook,1,President,NaN,DEM,Hillary Clinton / Tim Kaine,122,Crook-1
5,Crook,1,U.S. Senate,NaN,DEM,Ron Wyden,214,Crook-1
6,Crook,1,President,NaN,PGP,Jill Stein / Ajamu Baraka,8,Crook-1
...,...,...,...,...,...,...,...,...
72402,Sherman,3,State Treasurer,NaN,NaN,Under Votes,29,Sherman-3
72403,Sherman,4,State Treasurer,NaN,NaN,Over Votes,0,Sherman-4
72404,Sherman,4,State Treasurer,NaN,NaN,Under Votes,12,Sherman-4
72405,Sherman,5,State Treasurer,NaN,NaN,Over Votes,0,Sherman-5


In [49]:
election_results = election_results[~election_results["candidate"].isin(['Over Votes','Under Votes','Total'])]
print(election_results["candidate"].unique())

['Steven C Reynolds' 'Donald J Trump / Mike Pence'
 'Hillary Clinton / Tim Kaine' 'Ron Wyden' 'Jill Stein / Ajamu Baraka'
 'Mark Callahan' 'Gary Johnson / Bill Weld' 'Eric Navickas' 'Jim Lindsay'
 'Shanti S Lewallen' 'Write-ins' 'James Foster' 'Lars D H Hedbor'
 'Sharon L Durbin' 'Chris Henry' 'Ellen Rosenblum' 'Kate Brown'
 'Tobias Read' 'Dennis Richardson' 'Cliff Thomason' 'Paul Damian Wells'
 'Daniel Zene Crowe' 'Jeff Gudman' 'Aaron Donald Auer' 'Brad Avakian'
 'Chris Telfer' 'Alan Zundel' 'Bud Pierce' 'Michael Marsh'
 'Donald J Trump' 'Hillary Clinton' 'Jill Stein' 'Gary Johnson'
 'Donald Trump/Mike Pence' 'Gary Johnson/Bill Weld'
 'Hillary Clinton/Tim Kaine' 'Jill Stein/Ajamu Baraka' 'Donald Trump'
 'Steven Reynolds' 'Shanti Lewallen' 'Aaron Auer' 'Sharon Durbin'
 'Paul Wells' 'Lars Hedbor' 'Daniel Crowe']


In [50]:
print(election_results["pivot_id"])
election_results["cand-race"] = election_results["office"].astype(str) +"-"+ election_results["candidate"].astype(str)
print(election_results["cand-race"])

0          Crook-1
1          Crook-1
3          Crook-1
5          Crook-1
6          Crook-1
           ...    
72342    Sherman-5
72343    Sherman-5
72344    Sherman-5
72345    Sherman-5
72346    Sherman-5
Name: pivot_id, Length: 46070, dtype: object
0                U.S. Senate-Steven C Reynolds
1        President-Donald J Trump / Mike Pence
3        President-Hillary Clinton / Tim Kaine
5                        U.S. Senate-Ron Wyden
6          President-Jill Stein / Ajamu Baraka
                         ...                  
72342                U.S. Senate-Mark Callahan
72343                U.S. Senate-Eric Navickas
72344                  U.S. Senate-Jim Lindsay
72345            U.S. Senate-Shanti S Lewallen
72346                    U.S. Senate-Write-ins
Name: cand-race, Length: 46070, dtype: object


In [51]:
election_results_pivot = pd.pivot_table(election_results,values=["votes"],index=["pivot_id"],columns=["cand-race"],aggfunc='sum')

In [52]:
election_results_pivot.columns = election_results_pivot.columns.droplevel(0)

In [53]:
election_results_pivot.columns

Index(['Attorney General-Daniel Crowe', 'Attorney General-Daniel Zene Crowe',
       'Attorney General-Ellen Rosenblum', 'Attorney General-Lars D H Hedbor',
       'Attorney General-Lars Hedbor', 'Attorney General-Write-ins',
       'Governor-Aaron Auer', 'Governor-Aaron Donald Auer',
       'Governor-Bud Pierce', 'Governor-Cliff Thomason',
       'Governor-James Foster', 'Governor-Kate Brown', 'Governor-Write-ins',
       'President-Donald J Trump', 'President-Donald J Trump / Mike Pence',
       'President-Donald Trump', 'President-Donald Trump/Mike Pence',
       'President-Gary Johnson', 'President-Gary Johnson / Bill Weld',
       'President-Gary Johnson/Bill Weld', 'President-Hillary Clinton',
       'President-Hillary Clinton / Tim Kaine',
       'President-Hillary Clinton/Tim Kaine', 'President-Jill Stein',
       'President-Jill Stein / Ajamu Baraka',
       'President-Jill Stein/Ajamu Baraka', 'President-Write-ins',
       'Secretary of State-Alan Zundel', 'Secretary of State

In [54]:
{'Attorney General-Daniel Crowe':"", 
'Attorney General-Daniel Zene Crowe':"",
'Attorney General-Ellen Rosenblum':"", 
'Attorney General-Lars D H Hedbor':"",
'Attorney General-Lars Hedbor':"",
'Attorney General-Write-ins':"",

'Governor-Aaron Auer':"",
'Governor-Aaron Donald Auer':"",
'Governor-Bud Pierce':"",
'Governor-Cliff Thomason':"",
'Governor-James Foster':"",
'Governor-Kate Brown':"",
'Governor-Write-ins':"",

'President-Donald J Trump':"G16PRERTRU", 
'President-Donald J Trump / Mike Pence':"G16PRERTRU",
'President-Donald Trump':"G16PRERTRU", 
'President-Donald Trump/Mike Pence':"G16PRERTRU",
'President-Gary Johnson':"G16PRELJOH", 
'President-Gary Johnson / Bill Weld':"G16PRELJOH",
'President-Gary Johnson/Bill Weld':"G16PRELJOH",
'President-Hillary Clinton':"G16PREDCLI",
'President-Hillary Clinton / Tim Kaine':"G16PREDCLI",
'President-Hillary Clinton/Tim Kaine':"G16PREDCLI", 
'President-Jill Stein':"G16PREGSTE",
'President-Jill Stein / Ajamu Baraka':"G16PREGSTE",
'President-Jill Stein/Ajamu Baraka':"G16PREGSTE", 
'President-Write-ins':"G16PREOWRI",

'Secretary of State-Alan Zundel':"G16SOSGZUN", 
'Secretary of State-Brad Avakian':"G16SOSDAVA",
'Secretary of State-Dennis Richardson':"G16SOSRRIC",
'Secretary of State-Michael Marsh':"G16SOSCMAR", 
'Secretary of State-Paul Damian Wells':"G16SOSIWEL", 
'Secretary of State-Paul Wells':"G16SOSIWEL",
'Secretary of State-Sharon Durbin':"G16SOSLDUR",
'Secretary of State-Sharon L Durbin':"G16SOSLDUR",
'Secretary of State-Write-ins':"G16SOSOWRI",

'State Treasurer-Chris Henry':"G16TREPHEN",
'State Treasurer-Chris Telfer':"G16TREITEL",
'State Treasurer-Jeff Gudman':"G16TRERGUD",
'State Treasurer-Tobias Read':"G16TREDREA",  
'State Treasurer-Write-ins':"G16TREOWRI",

'U.S. Senate-Eric Navickas':"G16USSGNAV",
'U.S. Senate-Jim Lindsay':"G16USSLLIN",
'U.S. Senate-Mark Callahan':"G16USSRCAL", 
'U.S. Senate-Ron Wyden':"G16USSDWYD", 
'U.S. Senate-Shanti Lewallen':"G16USSWLEW",
'U.S. Senate-Shanti S Lewallen':"G16USSWLEW", 
'U.S. Senate-Steven C Reynolds':"G16USSIREY",
'U.S. Senate-Steven Reynolds':"G16USSIREY", 
'U.S. Senate-Write-ins':"G16USSOWRI"}

{'Attorney General-Daniel Crowe': '',
 'Attorney General-Daniel Zene Crowe': '',
 'Attorney General-Ellen Rosenblum': '',
 'Attorney General-Lars D H Hedbor': '',
 'Attorney General-Lars Hedbor': '',
 'Attorney General-Write-ins': '',
 'Governor-Aaron Auer': '',
 'Governor-Aaron Donald Auer': '',
 'Governor-Bud Pierce': '',
 'Governor-Cliff Thomason': '',
 'Governor-James Foster': '',
 'Governor-Kate Brown': '',
 'Governor-Write-ins': '',
 'President-Donald J Trump': 'G16PRERTRU',
 'President-Donald J Trump / Mike Pence': 'G16PRERTRU',
 'President-Donald Trump': 'G16PRERTRU',
 'President-Donald Trump/Mike Pence': 'G16PRERTRU',
 'President-Gary Johnson': 'G16PRELJOH',
 'President-Gary Johnson / Bill Weld': 'G16PRELJOH',
 'President-Gary Johnson/Bill Weld': 'G16PRELJOH',
 'President-Hillary Clinton': 'G16PREDCLI',
 'President-Hillary Clinton / Tim Kaine': 'G16PREDCLI',
 'President-Hillary Clinton/Tim Kaine': 'G16PREDCLI',
 'President-Jill Stein': 'G16PREGSTE',
 'President-Jill Stein / Aj

In [ ]:

G16GOVDBRO - Kate Brown (Democratic Party and Working Families Party (fusion candidate))
G16GOVRPIE - Bud Pierce (Republican Party)
G16GOVLFOS - James Foster (Libertarian Party)
G16GOVCAUE - Aaron Donald Auer (Constitution Party)
G16GOVITHO - Cliff Thomason (Independent Party)
G16GOVOWRI - Write-in Votes

G16ATGDROS - Ellen Rosenblum (Democratic Party and Working Families Party (fusion candidate))
G16ATGRCRO - Daniel Zene Crowe (Republican Party)
G16ATGLHED - Lars D. H. Hedbor (Libertarian Party)
G16ATGOWRI - Write-in Votes



